In [1]:
import gradio as gr
import pandas as pd

# Language-specific texts for both inputs and output
languages = {
    "English": {
        "upload": "Upload Excel or CSV File",
        "product": "Enter Product Name",
        "expected": "Enter Customer's Expected Price (INR)",
        "submit": "Find Best Price",
        "no_product": "❌ Product not found.",
        "no_column": "❌ Product name column not found.",
        "closest_price": "✅ Closest Price ≤ Expected",
        "no_match": "❌ No matching price found under expected value.",
        "best_price": "💰 Best Price Overall",
        "error": "❌ Error:",
        "product": "🧾 Product",
        "expected_price": "🎯 Expected Price",
        "prices_from_platforms": "💸 Prices from platforms:"
    },
    "Hindi": {
        "upload": "एक्सेल या CSV फ़ाइल अपलोड करें",
        "product": "उत्पाद का नाम दर्ज करें",
        "expected": "ग्राहक द्वारा अपेक्षित मूल्य दर्ज करें (INR)",
        "submit": "सर्वश्रेष्ठ मूल्य खोजें",
        "no_product": "❌ उत्पाद नहीं मिला।",
        "no_column": "❌ उत्पाद शीर्षक कॉलम नहीं मिला।",
        "closest_price": "✅ अपेक्षित मूल्य से कम या बराबर निकटतम उपलब्ध मूल्य",
        "no_match": "❌ कोई प्लेटफ़ॉर्म इस मूल्य पर उत्पाद नहीं दे रहा।",
        "best_price": "💰 सर्वोत्तम मूल्य",
        "error": "❌ त्रुटि:",
        "product": "🧾 उत्पाद",
        "expected_price": "🎯 अपेक्षित मूल्य",
        "prices_from_platforms": "💸 प्लेटफ़ॉर्म से कीमतें:"
    },
    "Telugu": {
        "upload": "Excel లేదా CSV ఫైల్ అప్‌లోడ్ చేయండి",
        "product": "ఉత్పత్తి పేరును నమోదు చేయండి",
        "expected": "గ్రాహక అంచనా ధరను నమోదు చేయండి (INR)",
        "submit": "ఉత్తమ ధర కనుగొనండి",
        "no_product": "❌ ఉత్పత్తి కనపడలేదు.",
        "no_column": "❌ ఉత్పత్తి శీర్షిక కాలమ్ కనబడలేదు.",
        "closest_price": "✅ అంచనా ధర కంటే తక్కువ లేదా సమానం ఉన్న అత్యంత సమీప ధర",
        "no_match": "❌ ఈ ధరకు ఉత్పత్తి లేదు.",
        "best_price": "💰 ఉత్తమ ధర",
        "error": "❌ లోపం:",
        "product": "🧾 ఉత్పత్తి",
        "expected_price": "🎯 అంచనా ధర",
        "prices_from_platforms": "💸 ప్లాట్‌ఫారమ్‌ల నుండి ధరలు:"
    },
    "Kannada": {
        "upload": "ಎಕ್ಸೆಲ್ ಅಥವಾ CSV ಫೈಲ್ ಅಪ್ಲೋಡ್ ಮಾಡಿ",
        "product": "ಉತ್ಪನ್ನದ ಹೆಸರನ್ನು ನಮೂದಿಸಿ",
        "expected": "ಗ್ರಾಹಕ ನಿರೀಕ್ಷಿತ ಬೆಲೆಯನ್ನು ನಮೂದಿಸಿ (INR)",
        "submit": "ಉತ್ತಮ ಬೆಲೆಯನ್ನು ಹುಡುಕಿ",
        "no_product": "❌ ಉತ್ಪನ್ನ ಕಂಡುಬಂದಿಲ್ಲ.",
        "no_column": "❌ ಉತ್ಪನ್ನ ಶೀರ್ಷಿಕೆ ಕಾಲಮ್ ಕಂಡುಬಂದಿಲ್ಲ.",
        "closest_price": "✅ ನಿರೀಕ್ಷಿತ ಬೆಲೆಗೆ ಸಮಾನ ಅಥವಾ ಕಡಿಮೆ ಹೊಂದಿರುವ ಅತ್ಯಂತ ಹತ್ತಿರದ ಬೆಲೆ",
        "no_match": "❌ ಈ ಬೆಲೆಗೆ ಯಾವುದೇ ವೇದಿಕೆಯಲ್ಲಿ ಉತ್ಪನ್ನ ಲಭ್ಯವಿಲ್ಲ.",
        "best_price": "💰 ಒಟ್ಟೂ ಉತ್ತಮ ಬೆಲೆ",
        "error": "❌ ದೋಷ:",
        "product": "🧾 ಉತ್ಪನ್ನ",
        "expected_price": "🎯 ನಿರೀಕ್ಷಿತ ಬೆಲೆ",
        "prices_from_platforms": "💸 ವೇದಿಕೆಗಳಿಂದ ಬೆಲೆಗಳು:"
    }
}

# Process file function
def process_file(file, product_name, expected_price, lang):
    text = languages.get(lang, languages["English"])
    try:
        df = pd.read_excel(file.name) if file.name.endswith('.xlsx') else pd.read_csv(file.name)
        df.columns = [c.lower().strip() for c in df.columns]
        title_col = next((c for c in df.columns if 'title' in c or 'product' in c), None)
        if not title_col:
            return text["no_column"]

        matches = df[df[title_col].astype(str).str.contains(product_name, case=False, na=False)]
        if matches.empty:
            return text["no_product"]

        row = matches.iloc[0]

        platforms = {
            "Meesho": next((c for c in df.columns if "meesho" in c), None),
            "Myntra": next((c for c in df.columns if "myntra" in c), None),
            "Amazon": next((c for c in df.columns if "amazon" in c), None),
            "Flipkart": next((c for c in df.columns if "flipkart" in c), None),
            "Ajio": next((c for c in df.columns if "ajio" in c), None),
        }

        prices = {}
        best_price = float('inf')
        best_platform = None
        closest_price = None
        closest_diff = float('inf')

        for platform, col in platforms.items():
            if col:
                try:
                    price = float(row[col])
                    prices[platform] = price
                    if price < best_price:
                        best_price = price
                        best_platform = platform
                    if price <= expected_price and (expected_price - price) < closest_diff:
                        closest_price = (platform, price)
                        closest_diff = expected_price - price
                except:
                    prices[platform] = "N/A"
            else:
                prices[platform] = "N/A"

        output = f"{text['product']}: *{row[title_col]}*\n\n"
        output += f"{text['expected_price']}: ₹{expected_price:.2f}\n\n"
        output += f"{text['prices_from_platforms']}\n"
        for p, val in prices.items():
            output += f"- {p}: ₹{val if val != 'N/A' else 'N/A'}\n"

        if closest_price:
            output += f"\n{text['closest_price']}: ₹{closest_price[1]:.2f} on {closest_price[0]}"
        else:
            output += f"\n{text['no_match']}"

        if best_platform:
            output += f"\n{text['best_price']}: ₹{best_price:.2f} on {best_platform}"

        return output
    except Exception as e:
        return f"{text['error']} {str(e)}"

# Update labels dynamically
def update_labels(language):
    text = languages.get(language, languages["English"])
    return (
        gr.update(label=text["upload"]),
        gr.update(label=text["product"]),
        gr.update(label=text["expected"]),
        gr.update(value=text["submit"])
    )

# Build Gradio Interface
with gr.Blocks() as demo:
    lang_selector = gr.Dropdown(
        choices=list(languages.keys()),
        value="English",
        label="🌐 Select Language"
    )

    file_input = gr.File(label="Upload Excel or CSV File")
    product_input = gr.Textbox(label="Enter Product Name")
    expected_price_input = gr.Number(label="Enter Customer's Expected Price (INR)")
    submit_button = gr.Button("Find Best Price")
    output_display = gr.Markdown()

    lang_selector.change(
        update_labels,
        inputs=[lang_selector],
        outputs=[file_input, product_input, expected_price_input, submit_button]
    )

    submit_button.click(
        fn=process_file,
        inputs=[file_input, product_input, expected_price_input, lang_selector],
        outputs=output_display
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
